## Prediction of Sales

#### Import Important Framework

In [108]:
import os
import warnings
import sys
from datetime import datetime

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
#from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
#import mlflow
#import mlflow.sklearn
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import pickle

##### Import modules

In [109]:
sys.path.append(os.path.abspath(os.path.join('../Scripts')))
from plotting import Plotting
from preprocessing import Preprocess
from file import Myfile

Instantiate modules

In [110]:
preprocess = Preprocess()
plot = Plotting()
file = Myfile()

Load data from dvc

In [111]:
df = file.read_csv('../data/train_processed.csv')

C:\Users\user\Desktop\10Academy\Rossmann-Pharmaceuticals-Sales-prediction\Scripts\file.py:11: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


--> file read as csv


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Unnamed: 0                 1017209 non-null  int64  
 1   Date                       1017209 non-null  object 
 2   Store                      1017209 non-null  int64  
 3   DayOfWeek                  1017209 non-null  int64  
 4   Sales                      1017209 non-null  float64
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   CompetitionDistance        1017209 non-null  float64
 10  CompetitionOpenSinceMonth  1017209 non-null  float64
 11  CompetitionOpenSinceYear   1017209 non-null  float64
 12  Promo2                     1017209 non-null  int64  
 13  Promo2SinceW

In [114]:
df.drop(['StateHoliday'], axis=1, inplace=True)

In [115]:
# Drop rows that are closed (close=0) & sales=0
df = df[df['Open'] == 1]
df = df[df['Sales'] > 0.0]

In [117]:
## Lable Date and StateHoiday Train
columns = ['Sales','Store', 'DayOfWeek', 'Open', 'Promo', 'CompetitionDistance',  'SchoolHoliday', 'Day', 'weekofyear','Month', 'Year', 'StoreType',
              'Assortment', 'Promo2']

feature = ['Store', 'DayOfWeek', 'Open', 'Promo', 'CompetitionDistance', 'SchoolHoliday', 'Day', 'weekofyear','Month', 'Year', 'StoreType',
              'Assortment', 'Promo2']

sample_size = 1000
sampled_df = df[columns].sample(sample_size)

In [118]:
## Separet Test and train files
train_x = sampled_df[feature]
train_y = sampled_df[['Sales']]
train_x.head()

,Store,DayOfWeek,Open,Promo,CompetitionDistance,SchoolHoliday,Day,weekofyear,Month,Year,StoreType,Assortment,Promo2
567105,356,5,1,0,8970.0,0,8,6,2,2014,3,2,1
156277,178,4,1,0,1710.0,0,13,11,3,2015,3,0,0
978028,959,1,1,1,1060.0,1,5,6,2,2013,0,2,0
211716,982,4,1,0,21930.0,0,23,4,1,2015,3,0,0
986096,107,0,1,0,1930.0,0,28,5,1,2013,0,0,0


#### Label Encoding

In [119]:
### Label encoding
lb = LabelEncoder()
train_x['StoreType'] = lb.fit_transform(train_x['StoreType'])
train_x['Assortment'] = lb.fit_transform(train_x['Assortment'])

train_x.info()

C:\Users\user\AppData\Local\Temp\ipykernel_7848\3609816834.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['StoreType'] = lb.fit_transform(train_x['StoreType'])
C:\Users\user\AppData\Local\Temp\ipykernel_7848\3609816834.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x['Assortment'] = lb.fit_transform(train_x['Assortment'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 567105 to 781612
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Store                1000 non-null   int64  
 1   DayOfWeek            1000 non-null   int64  
 2   Open                 1000 non-null   int64  
 3   Promo                1000 non-null   int64  
 4   CompetitionDistance  1000 non-null   float64
 5   SchoolHoliday        1000 non-null   int64  
 6   Day                  1000 non-null   int64  
 7   weekofyear           1000 non-null   int64  
 8   Month                1000 non-null   int64  
 9   Year                 1000 non-null   int64  
 10  StoreType            1000 non-null   int64  
 11  Assortment           1000 non-null   int64  
 12  Promo2               1000 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 109.4 KB


#### Scale data

In [121]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X = train_x
Y = y_scaler.fit_transform(train_y)

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [123]:
randomferest_reg = RandomForestRegressor(n_estimators = 100, max_depth=25, random_state=0)
randomferest_reg.fit(X_train, y_train)

C:\Users\user\AppData\Local\Temp\ipykernel_7848\193663102.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  randomferest_reg.fit(X_train, y_train)


RandomForestRegressor(max_depth=25, random_state=0)

In [125]:
score = randomferest_reg.score(X_test, y_test)
print(f"Prediction Score of the Model is {round(score * 100, 2)}%")

Prediction Score of the Model is 22.29%


In [126]:
def train_model(X, Y, model_type='LinearRegression'):
    model = LinearRegression()
    
    if(model_type == 'RandomForest'):
        model = RandomForestRegressor(n_estimators = 100, max_depth=5, random_state=0)
    
        
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X):
        with mlflow.start_run(run_name=model_type, nested = True):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]

            model.fit(X_train, y_train)
            score = model.score(X_test, y_test)
            print(f"Prediction Score of {model_type} : {round(score * 100, 2)}%")
            
            if(model_type == 'LinearRegression'):
                mlflow.log_param('LR-Score', score)
                
            if(model_type == 'RandomForest'):
                mlflow.log_param('RF-Score', score)
                
            mlflow.sklearn.log_model(model, model_type +' Model')
            mlflow.end_run()
            
    print()

#### Building models with sklearn pipelines

In [127]:
## Prepare the training and test features
x = train_x
y = train_y

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [128]:
### Pipe Lines ###

rfr = Pipeline(steps=[
    ('scaler1', MinMaxScaler()),
    ('rfr_regretion', RandomForestRegressor(n_estimators = 100, max_depth=5, random_state=0))
])

lr = Pipeline(steps=[
    ('scaler2', MinMaxScaler()),
    ('lr_regretion', LinearRegression())
])

dt = Pipeline(steps=[
    ('scaler3', MinMaxScaler()),
    ('dt_regretion', DecisionTreeRegressor(random_state = 0)  )
])


In [129]:
rfr.fit(X_train, y_train)
lr.fit(X_train, y_train)
dt.fit(X_train, y_train)

C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py:394: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('scaler3', MinMaxScaler()),
                ('dt_regretion', DecisionTreeRegressor(random_state=0))])

In [130]:
rfr_score = rfr.score(X_test, y_test)
lr_score = lr.score(X_test, y_test)
dt_score = dt.score(X_test, y_test)

print(f"Prediction Score of RandomForestRegretion : {round(rfr_score * 100, 2)}%")
print(f"Prediction Score of LinearRegretion : {round(lr_score * 100, 2)}%")
print(f"Prediction Score of DecisionTree : {round(dt_score * 100, 2)}%")


Prediction Score of RandomForestRegretion : 16.51%
Prediction Score of LinearRegretion : 18.37%
Prediction Score of DecisionTree : -34.31%


#### Choose a loss function

In [131]:
rfr_prediction = rfr.predict(X_test)
lr_prediction = lr.predict(X_test)
dt_prediction = dt.predict(X_test)

rfr_rmse, rft_mae, _ = preprocess.eval_metrics(y_test, rfr_prediction)
lr_rmse, lr_mae, _ = preprocess.eval_metrics(y_test, lr_prediction)
dt_rmse, dt_mae, _ = preprocess.eval_metrics(y_test, dt_prediction)


# print(f"Mean Absolute Loss : {round(mae * 100, 2)}%")
print(f"RandomForest Mean Square Loss : {rfr_rmse}")
print(f"Linear Mean Square Loss : {lr_rmse}")
print(f"DecisionTree Mean Square Loss : {dt_rmse }%")

AttributeError: 'Preprocess' object has no attribute 'eval_metrics'

#### Post Prediction analysis

In [132]:
important_features = rfr[1].feature_importances_
plot.bar(feature_columns, important_features, 'RandomForestRegretion Feature Importance')

TypeError: bar() takes from 2 to 3 positional arguments but 4 were given

In [133]:
important_features = lr[1].coef_[0]
from matplotlib import pyplot as plt
plot.bar(feature_columns, list(important_features), "LinearRegretion Feature Importance")

TypeError: bar() takes from 2 to 3 positional arguments but 4 were given

In [134]:
important_features = dt[1].feature_importances_
plot.bar(feature_columns, important_features, 'tg')

TypeError: bar() takes from 2 to 3 positional arguments but 4 were given

#### Serialize models

In [135]:
# Save model based on time st
pickle.dump(rfr, open('../models/''random forest-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.pkl', 'wb'))
pickle.dump(lr, open('../models/''linearregression-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.pkl', 'wb'))
pickle.dump(dt, open('../models/''Decision tree-' + datetime.now().strftime("%Y-%m-%d-%H-%M-%S") + '.pkl', 'wb'))